In [2]:
import onshape as brepio
from mechanical.visualize2 import plot_assembly, filter_assembly
import os
import pandas as ps
import ipywidgets as widgets
from ipywidgets.embed import embed_minimal_html
from IPython.display import clear_output
import importlib
from IPython.display import display

In [47]:
import numpy as np

In [3]:
df_name = '/fast/jamesn8/assembly_data/assembly_data_with_transforms_all.h5'
df_name_part = '/fast/jamesn8/assembly_data/assembly_data_with_transforms_all.h5_segmentation.h5'
assembly_df = ps.read_hdf(df_name,'assembly')
part_df = ps.read_hdf(df_name_part,'part')
mate_df = ps.read_hdf(df_name,'mate')

In [77]:
mate_df_by_ass = mate_df.copy()
mate_df_by_ass = mate_df.set_index('Assembly')
mate_df_by_ass['MateIndex'] = mate_df.index

In [57]:
part_df.loc[18][lambda df: df['PartOccurrenceID'] == 'M8+41AMzQm6OVY2kU']

,PartOccurrenceID,did,mv,eid,config,PartId,Transform,HasGeometry,RigidComponentID
Assembly,,,,,,,,,
18,M8+41AMzQm6OVY2kU,b6c979cf4e6660ff81e5a9c2,67d2f776871477086fbc37c5,1520c99d5029db1cafd9d6e5,default,JHD,"[[5.551115e-17, -1.0, -1.2143064e-16, 0.021928...",True,1


In [5]:
path_to_index = ps.Series(dict((v,k) for k,v in assembly_df['AssemblyPath'].iteritems()))

In [6]:
has_geometry = part_df.groupby('Assembly')['HasGeometry'].agg(all)
assembly_df['HasAllGeometry'] = has_geometry

In [7]:
part_df.set_index('Assembly', inplace=True)

In [8]:
datapath = '/projects/grail/benjones/cadlab'

In [9]:
loader = brepio.Loader(datapath)

In [80]:
mate_types = ['PIN_SLOT', 'BALL', 'PARALLEL', 'SLIDER', 'REVOLUTE', 'CYLINDRICAL', 'PLANAR', 'FASTENED']
@widgets.interact(sample="7492")
def display_sample(sample):
    try:
        sample = int(sample)
    except ValueError:
        sample = path_to_index[sample]
    assemblypath = assembly_df.loc[sample, "AssemblyPath"]
    did, mv, eid = assemblypath.split('_')
    print(f'https://cad.onshape.com/documents/{did}/w/8a6b7ec29c9e253f04b1a1fb/m/{mv}/e/{eid}')
    print(assemblypath, f' ({sample})')
    try:
        geo, mates = loader.load_flattened(assemblypath + '.json', skipInvalid=True)
        mates = [mate for mate in mates if len(mate.matedEntities) == 2]
    except FileNotFoundError as e:
        print(f'File not found: {e}')
        return
    assert(list(geo) == list(part_df.loc[sample,'PartOccurrenceID']))
    mate_counts = dict()
    for mate in mates:
        if len(mate.matedEntities) == 2:
            if mate.type not in mate_counts:
                mate_counts[mate.type] = 0
            mate_counts[mate.type] += 1
    num_connected = assembly_df.loc[sample, "ConnectedComponents"]
    num_rigid = assembly_df.loc[sample, "RigidPieces"]
    if num_connected > 1:
        print('warning:',num_connected,'connected components')
    print('rigid pieces:',num_rigid)
    print('total parts:',len(geo))
    print(f'mates: {len(mates)}: ',mate_counts)

    mate_subset = mate_df_by_ass.loc[sample]
    geo_to_id = {occ: i for i,occ in enumerate(geo)}
    print(geo_to_id)
    print([(len(mate.matedEntities),mate.type) for mate in mates])
    pairs = [(geo_to_id[mate.matedEntities[0][0]],geo_to_id[mate.matedEntities[1][0]]) for mate in mates]
    choices = [(f'mate ({pairs[i][0]},{pairs[i][1]}) (ID {mate_subset.iloc[i]["MateIndex"]}) ({mates[i].type}) ({mates[i].name})',i) for i in range(len(mates)) if len(mates[i].matedEntities) == 2]
    choices = [('fullAssembly', -1)] + choices
    @widgets.interact(mate=choices)
    def display_subset(mate):
        rigid_labels = list(part_df.loc[sample, 'RigidComponentID'])
        nonlocal geo, mates
        if mate >= 0:
            print(pairs[mate])
            geo_filtered, mates_filtered = filter_assembly(geo, mates, pairs[mate])
            rigid_labels = [r for i,r in enumerate(rigid_labels) if i in pairs[mate]]
        else:
            geo_filtered = geo
            mates_filtered = mates
        badOccs = [k for k in geo if geo[k][1] is None or geo[k][1].V.shape[0] == 0]
        if len(badOccs) > 0:
            print(f'warning: {len(badOccs)} invalid parts!')
        #for o in badOccs:
        #    geo.pop(o)
        renderer = plot_assembly(geo_filtered, mates_filtered, rigid_labels = rigid_labels if mate < 0 else None)
        display(renderer)
        #embed_minimal_html('export.html', views=[renderer], title='Viewer export')
            #print('num mates:',len(mates))
    

interactive(children=(Text(value='7492', description='sample'), Output()), _dom_classes=('widget-interact',))

In [62]:
part_df.keys()

Index(['PartOccurrenceID', 'did', 'mv', 'eid', 'config', 'PartId', 'Transform',
       'HasGeometry', 'RigidComponentID'],
      dtype='object')